# SUMMARY 
This notebook is responsible for the basic data collection and preparation. The dataset we use comes from the ECDC and provides day-by-day accounts of cases and deaths for each affected country. This data file is updated each dat and so needs to be downloaded to keep it up to date.

We do some basic data processing to convert daily numbers into cumulative numbers etc. The processed dataframe is then saved to file for later use.

In [74]:
from datetime import date

import pandas as pd
import numpy as np


# Download the Raw Dataset

Switched to using a daily dataset from https://www.ecdc.europa.eu/en/novel-coronavirus-china
The main reason for this is that this dataset goes back to 31/12/2019 whereas the JH dataset starts about a month later when China already had 500+ cases and 17 deaths.

In [75]:
# The download file template (encodes the current day's date)
ecdc_dataset_raw = 'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide{}.xlsx'

# Save the processed data to here.
ecdc_dataset_processed = '../data/processed/ecdc_dataset.csv'

In [76]:
# Create the filename for tpday's dataset and download it.
todays_date = "-{}-{:02}-{:02}".format(date.today().year, date.today().month, date.today().day)

ecdc_dataset_raw.format(todays_date)

'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-26.xlsx'

In [77]:
df = pd.read_excel(ecdc_dataset_raw.format(todays_date))
df.shape

(6931, 10)

In [78]:
df

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,Country Code,Pop_Data.2018
0,2020-03-26,26,3,2020,33,0,Afghanistan,AF,AFG,37172386.0
1,2020-03-25,25,3,2020,2,0,Afghanistan,AF,AFG,37172386.0
2,2020-03-24,24,3,2020,6,1,Afghanistan,AF,AFG,37172386.0
3,2020-03-23,23,3,2020,10,0,Afghanistan,AF,AFG,37172386.0
4,2020-03-22,22,3,2020,0,0,Afghanistan,AF,AFG,37172386.0
...,...,...,...,...,...,...,...,...,...,...
6926,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
6927,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
6928,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
6929,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


In [79]:
# Change the column names.
df.columns = ['date', 'day', 'month', 'year', 'cases', 'deaths', 'country', 'id', 'code', 'pop']

# Add a number of days since records began column.
df['day'] = (df['date'] - df['date'].min()).map(lambda d: d.days)

# Sort by day, oldest first
df = df.sort_values(by='day', ascending=True).set_index('day')


In [80]:
df.country.unique(), df.country.nunique()

(array(['New_Zealand', 'Philippines', 'Ireland', 'India', 'San_Marino',
        'South_Korea', 'Ecuador', 'Netherlands', 'Germany', 'Sweden',
        'Luxembourg', 'Pakistan', 'Spain', 'Malaysia', 'Egypt', 'Russia',
        'Switzerland', 'Czech_Republic', 'Estonia', 'Taiwan', 'Greece',
        'Japan', 'United_Kingdom', 'Romania', 'Bahrain',
        'United_Arab_Emirates',
        'Cases_on_an_international_conveyance_Japan', 'Australia',
        'Canada', 'Iceland', 'Nigeria', 'Italy', 'Mexico', 'Kuwait',
        'Iran', 'Azerbaijan', 'Oman', 'Qatar', 'Indonesia', 'Cambodia',
        'Georgia', 'Singapore', 'France', 'Thailand', 'Denmark', 'Israel',
        'Lebanon', 'Sri_Lanka', 'Vietnam', 'Monaco', 'Brazil', 'Belarus',
        'Norway', 'Dominican_Republic', 'Lithuania', 'Finland', 'Armenia',
        'United_States_of_America', 'North_Macedonia', 'Nepal', 'Algeria',
        'Iraq', 'Afghanistan', 'Belgium', 'China', 'Croatia', 'Austria',
        'Sint_Maarten', 'Latvia', 'Jordan',

In [81]:
# Canada is represented in title and Upper case. Looks like a data entry error.
df['country'] = np.where(df['country']=='CANADA', 'Canada', df['country'])

df.country.nunique()

192

In [82]:
df.head()

,date,month,year,cases,deaths,country,id,code,pop
day,,,,,,,,,
0,2019-12-31,12,2019,0,0,New_Zealand,NZ,NZL,4.885500e+06
0,2019-12-31,12,2019,0,0,Philippines,PH,PHL,1.066519e+08
0,2019-12-31,12,2019,0,0,Ireland,IE,IRL,4.853506e+06
0,2019-12-31,12,2019,0,0,India,IN,IND,1.352617e+09
0,2019-12-31,12,2019,0,0,San_Marino,SM,SMR,3.378500e+04


# Basic Data Processing
We will add a bunch of new columns to capture various features that we will need for future processing.

## Add Cumulative Totals for Cases and Deaths

In [83]:
df['cum_cases'] = df.groupby('country')['cases'].transform('cumsum')
df['cum_deaths'] = df.groupby('country')['deaths'].transform('cumsum')

## Calculate Daily %Change for Each Country

In [84]:
cum_cases_pct_change = df.groupby('country').apply(lambda g: g['cum_cases'].pct_change())
cum_deaths_pct_change = df.groupby('country').apply(lambda g: g['cum_deaths'].pct_change())

df = df.reset_index()\
        .set_index(['country', 'day'])\
        .join(cum_cases_pct_change, rsuffix='_pct_change')\
        .join(cum_deaths_pct_change, rsuffix='_pct_change')

df.sample()

,,date,month,year,cases,deaths,id,code,pop,cum_cases,cum_deaths,cum_cases_pct_change,cum_deaths_pct_change
country,day,,,,,,,,,,,,
Philippines,37,2020-02-06,2,2020,1,0,PH,PHL,106651922.0,3,1,0.5,0.0


In [85]:
df['cum_cases_pct_change'] = df['cum_cases_pct_change'].replace(np.inf, np.nan)
df['cum_deaths_pct_change'] = df['cum_deaths_pct_change'].replace(np.inf, np.nan)

df.shape

(6931, 12)

## Calculate Log10 Values for Cases and Deaths

In [86]:
df = df.reset_index().set_index('day')
df.sample()

,country,date,month,year,cases,deaths,id,code,pop,cum_cases,cum_deaths,cum_cases_pct_change,cum_deaths_pct_change
day,,,,,,,,,,,,,
45,Iraq,2020-02-14,2,2020,0,0,IQ,IRQ,38433600.0,0,0,NaN,NaN


In [87]:
cum_cases_log10 = df.groupby('country').apply(lambda g: g['cum_cases'].map(lambda n: np.log10(n) if n>0 else np.nan))
cum_deaths_log10 = df.groupby('country').apply(lambda g: g['cum_deaths'].map(lambda n: np.log10(n) if n>0 else np.nan))

df = df.reset_index()\
    .set_index(['country', 'day'])\
    .join(cum_cases_log10, rsuffix='_log10')\
    .join(cum_deaths_log10, rsuffix='_log10')

df.head()


,,date,month,year,cases,deaths,id,code,pop,cum_cases,cum_deaths,cum_cases_pct_change,cum_deaths_pct_change,cum_cases_log10,cum_deaths_log10
country,day,,,,,,,,,,,,,,
New_Zealand,0,2019-12-31,12,2019,0,0,NZ,NZL,4.885500e+06,0,0,NaN,NaN,NaN,NaN
Philippines,0,2019-12-31,12,2019,0,0,PH,PHL,1.066519e+08,0,0,NaN,NaN,NaN,NaN
Ireland,0,2019-12-31,12,2019,0,0,IE,IRL,4.853506e+06,0,0,NaN,NaN,NaN,NaN
India,0,2019-12-31,12,2019,0,0,IN,IND,1.352617e+09,0,0,NaN,NaN,NaN,NaN
San_Marino,0,2019-12-31,12,2019,0,0,SM,SMR,3.378500e+04,0,0,NaN,NaN,NaN,NaN


## Calculate Doubling Time
The doubling time represents the number of days it takes for cases/deaths to double. 

In [88]:
df = df.reset_index().set_index('day')
df.sample()

,country,date,month,year,cases,deaths,id,code,pop,cum_cases,cum_deaths,cum_cases_pct_change,cum_deaths_pct_change,cum_cases_log10,cum_deaths_log10
day,,,,,,,,,,,,,,,
79,Uruguay,2020-03-19,3,2020,29,0,UY,URY,3449299.0,79,0,0.58,NaN,1.897627,NaN


In [89]:
def doubling_time(s, d=5): return d*(np.log(2)/np.log(s/s.shift(d)))

cum_cases_dt = df.groupby('country').apply(lambda g: doubling_time(g['cum_cases']))

df = df.reset_index()\
    .set_index(['country', 'day'])\
    .join(cum_cases_dt, rsuffix='_dt')

df.sample()

,,date,month,year,cases,deaths,id,code,pop,cum_cases,cum_deaths,cum_cases_pct_change,cum_deaths_pct_change,cum_cases_log10,cum_deaths_log10,cum_cases_dt
country,day,,,,,,,,,,,,,,,
Armenia,7,2020-01-07,1,2020,0,0,AM,ARM,2951776.0,0,0,NaN,NaN,NaN,NaN,NaN


# Calculate Day Zero Days
To align the cases/deaths across different countries we choose a staring day based on when countries reach 100/10 cases/deaths.

In [90]:
df = df.reset_index().set_index('day')
df.sample()

,country,date,month,year,cases,deaths,id,code,pop,cum_cases,cum_deaths,cum_cases_pct_change,cum_deaths_pct_change,cum_cases_log10,cum_deaths_log10,cum_cases_dt
day,,,,,,,,,,,,,,,,
54,New_Zealand,2020-02-23,2,2020,0,0,NZ,NZL,4885500.0,0,0,NaN,NaN,NaN,NaN,NaN


In [91]:
def day_with_n_at_least_k(g, k): 
        
    # The zero_day offset
    d = g[g>=k].index.values[0] if g.max()>=k else np.nan
        
    return pd.Series(g.index.values-d, name='day', index=g.index.values)


min_cases, min_deaths = 100, 10


day_zero_for_cases = df.groupby('country').apply(
    lambda g: day_with_n_at_least_k(g['cum_cases'], min_cases)).reset_index()
day_zero_for_cases.columns = ['country', 'day', 'day_zero_cases']

day_zero_for_deaths = df.groupby('country').apply(
    lambda g: day_with_n_at_least_k(g['cum_deaths'], min_deaths)).reset_index()
day_zero_for_deaths.columns = ['country', 'day', 'day_zero_deaths']

df = df.reset_index()\
    .set_index(['country', 'day'])\
    .join(day_zero_for_cases.set_index(['country', 'day']))\
    .join(day_zero_for_deaths.set_index(['country', 'day']))\

    
df.sample()

,,date,month,year,cases,deaths,id,code,pop,cum_cases,cum_deaths,cum_cases_pct_change,cum_deaths_pct_change,cum_cases_log10,cum_deaths_log10,cum_cases_dt,day_zero_cases,day_zero_deaths
country,day,,,,,,,,,,,,,,,,,
Georgia,45,2020-02-14,2,2020,0,0,GE,GEO,3731000.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save Processed Datasets

In [92]:
df.reset_index().to_csv(ecdc_dataset_processed, index=False)

df.shape


(6931, 17)